In [27]:
# Librerías.
import pandas as pd
import Funciones as f

In [ ]:
%%capture
%run "5. Relleno de medianas de IP no asociados.ipynb"

In [ ]:
# PASO 1: REGISTRAR ESTADO ANTES Y CALCULAR MEDIANAS POR CATEGORÍA.
print("="*80)
print("VERIFICACIÓN GRANULAR: MEDIANAS POR CATEGORÍA")
print("="*80)

# Obtener todas las columnas IP_Item_X_Izq/Der.
Columnas_IP_Asociados = []
for Columna in list(dfs_Finales.values())[0].columns:
    if ('IP_Item_' in Columna and 
        ('_Izq_' in Columna or '_Der_' in Columna) and
        ('Respuesta' in Columna or 'Tiempo' in Columna)):
        Columnas_IP_Asociados.append(Columna)

# Diccionarios para almacenar datos ANTES del relleno.
Registro_Antes_Granular = {}
Medianas_Por_Categoria = {}

for Nombre_df, df in dfs_Finales.items():
    print(f"\n📋 REGISTRANDO ESTADO ANTES - {Nombre_df}")
    print("-" * 60)
    
    # 0. Verificar tipos de datos de las columnas IP.
    print("0. VERIFICANDO TIPOS DE DATOS:")
    Columnas_Problematicas = []
    for Columna in Columnas_IP_Asociados:
        if Columna in df.columns:
            Tipo_Datos = str(df[Columna].dtype)
            if Tipo_Datos == 'object':
                # Verificar si contiene strings numéricos.
                Muestra_Valores = df[Columna].dropna().head(10).tolist()
                print(f"   ⚠️  {Columna}: {Tipo_Datos} (Muestra: {Muestra_Valores})")
                Columnas_Problematicas.append(Columna)
            else:
                print(f"   ✅ {Columna}: {Tipo_Datos}")
    
    if Columnas_Problematicas:
        print(f"\n   🔧 Convirtiendo {len(Columnas_Problematicas)} columnas a numéricas...")
        for Columna in Columnas_Problematicas:
            df[Columna] = pd.to_numeric(df[Columna], errors='coerce')
        print("   ✅ Conversión completada")
    
    # 1. Calcular medianas por categoría para cada columna IP.
    print("1. CALCULANDO MEDIANAS POR CATEGORÍA:")
    Medianas_Por_Categoria[Nombre_df] = {}
    
    for Columna in Columnas_IP_Asociados:
        if Columna in df.columns:
            # Convertir la columna a numérica antes de calcular mediana.
            Columna_Numerica = pd.to_numeric(df[Columna], errors='coerce')
            Medianas_Columna = Columna_Numerica.groupby(df['Categoria_PASO_2023']).median()
            Medianas_Por_Categoria[Nombre_df][Columna] = Medianas_Columna.to_dict()
            
            print(f"   {Columna}:")
            for Categoria, Mediana in Medianas_Columna.items():
                if pd.notna(Mediana):
                    print(f"     {Categoria}: {Mediana}")
                else:
                    print(f"     {Categoria}: Sin datos válidos")
    
    # 2. Registrar todos los casos candidatos a relleno.
    print("\n2. IDENTIFICANDO CASOS CANDIDATOS A RELLENO:")
    
    Lista_Candidatos = []
    
    # Obtener pares únicos de Items.
    Items_Unicos = set()
    for Columna in Columnas_IP_Asociados:
        if '_Izq_' in Columna or '_Der_' in Columna:
            Partes = Columna.split('_')
            if len(Partes) >= 4:
                Numero_Item = Partes[2]
                Tipo_Variable = Partes[4]  # Respuesta o Tiempo.
                Items_Unicos.add((Numero_Item, Tipo_Variable))
    
    for Numero_Item, Tipo_Variable in Items_Unicos:
        Col_Izq = f'IP_Item_{Numero_Item}_Izq_{Tipo_Variable}'
        Col_Der = f'IP_Item_{Numero_Item}_Der_{Tipo_Variable}'
        
        if Col_Izq in df.columns and Col_Der in df.columns:
            
            # Casos donde Izq tiene valor y Der no (candidato a rellenar Der).
            Mask_Rellenar_Der = (df[Col_Izq].notna() & df[Col_Der].isna())
            
            for Indice in df[Mask_Rellenar_Der].index:
                Candidato = {
                    'ID': df.loc[Indice, 'ID'],
                    'Categoria': df.loc[Indice, 'Categoria_PASO_2023'],
                    'Item_Numero': Numero_Item,
                    'Tipo_Variable': Tipo_Variable,
                    'Columna_A_Rellenar': Col_Der,
                    'Direccion_A_Rellenar': 'Der',
                    'Valor_Pareja': df.loc[Indice, Col_Izq],
                    'Valor_Antes_Relleno': df.loc[Indice, Col_Der],  # Debería ser NaN.
                    'Mediana_Esperada': (Medianas_Por_Categoria[Nombre_df][Col_Der].get(df.loc[Indice, 'Categoria_PASO_2023']) 
                                        if Col_Der in Medianas_Por_Categoria[Nombre_df] 
                                        else None)
                }
                Lista_Candidatos.append(Candidato)
            
            # Casos donde Der tiene valor e Izq no (candidato a rellenar Izq).
            Mask_Rellenar_Izq = (df[Col_Der].notna() & df[Col_Izq].isna())
            
            for Indice in df[Mask_Rellenar_Izq].index:
                Candidato = {
                    'ID': df.loc[Indice, 'ID'],
                    'Categoria': df.loc[Indice, 'Categoria_PASO_2023'],
                    'Item_Numero': Numero_Item,
                    'Tipo_Variable': Tipo_Variable,
                    'Columna_A_Rellenar': Col_Izq,
                    'Direccion_A_Rellenar': 'Izq',
                    'Valor_Pareja': df.loc[Indice, Col_Der],
                    'Valor_Antes_Relleno': df.loc[Indice, Col_Izq],  # Debería ser NaN.
                    'Mediana_Esperada': (Medianas_Por_Categoria[Nombre_df][Col_Izq].get(df.loc[Indice, 'Categoria_PASO_2023']) 
                                        if Col_Izq in Medianas_Por_Categoria[Nombre_df] 
                                        else None)
                }
                Lista_Candidatos.append(Candidato)
    
    # Convertir a DataFrame para fácil manejo.
    df_Candidatos_Antes = pd.DataFrame(Lista_Candidatos)
    Registro_Antes_Granular[Nombre_df] = df_Candidatos_Antes
    
    print(f"   📊 Total de candidatos a relleno: {len(df_Candidatos_Antes)}")
    
    if len(df_Candidatos_Antes) > 0:
        print("   Distribución por Item:")
        Distribucion_Items = df_Candidatos_Antes.groupby(['Item_Numero', 'Tipo_Variable', 'Direccion_A_Rellenar']).size()
        for (Item, Tipo, Direccion), Cantidad in Distribucion_Items.items():
            print(f"     Item {Item} {Tipo} {Direccion}: {Cantidad} casos")
        
        print("   Distribución por Categoría:")
        Distribucion_Categorias = df_Candidatos_Antes['Categoria'].value_counts()
        for Categoria, Cantidad in Distribucion_Categorias.items():
            print(f"     {Categoria}: {Cantidad} casos")

print("\n" + "="*80)
print("✅ REGISTRO ANTES DEL RELLENO COMPLETADO")
print("="*80)

In [30]:
for Nombre_df, df in dfs_Finales.items():
   f.Rellenar_IP_Items_Asociados_Faltantes(df)

In [ ]:
# PASO 2: VERIFICAR CADA CASO INDIVIDUAL.
print("\n" + "="*80)
print("VERIFICACIÓN GRANULAR: CASO POR CASO")
print("="*80)

for Nombre_df, df in dfs_Finales.items():
    print(f"\n🔍 VERIFICANDO DATAFRAME: {Nombre_df}")
    print("-" * 60)
    
    df_Candidatos_Antes = Registro_Antes_Granular[Nombre_df]
    
    if len(df_Candidatos_Antes) == 0:
        print("   ℹ️  No había candidatos a relleno en este DataFrame")
        continue
    
    # Crear DataFrame de verificación.
    Lista_Verificacion = []
    
    for _, Candidato in df_Candidatos_Antes.iterrows():
        # Obtener el valor actual (después del relleno).
        Indice_Fila = df[df['ID'] == Candidato['ID']].index
        
        if len(Indice_Fila) > 0:
            Indice_Fila = Indice_Fila[0]
            Valor_Después_Relleno = df.loc[Indice_Fila, Candidato['Columna_A_Rellenar']]
            
            # Crear registro de verificación.
            Verificacion = {
                'ID': Candidato['ID'],
                'Categoria': Candidato['Categoria'],
                'Item_Numero': Candidato['Item_Numero'],
                'Tipo_Variable': Candidato['Tipo_Variable'],
                'Direccion_Rellenada': Candidato['Direccion_A_Rellenar'],
                'Columna_Rellenada': Candidato['Columna_A_Rellenar'],
                'Valor_Pareja': Candidato['Valor_Pareja'],
                'Valor_Antes_Relleno': Candidato['Valor_Antes_Relleno'],
                'Valor_Después_Relleno': Valor_Después_Relleno,
                'Mediana_Esperada': Candidato['Mediana_Esperada'],
                'Relleno_Exitoso': False,
                'Valor_Correcto': False,
                'Observaciones': ''
            }
            
            # Verificar si se realizó el relleno.
            if pd.isna(Candidato['Valor_Antes_Relleno']) and pd.notna(Valor_Después_Relleno):
                Verificacion['Relleno_Exitoso'] = True
                
                # Verificar si el valor es correcto.
                if (pd.notna(Candidato['Mediana_Esperada']) and 
                    abs(Valor_Después_Relleno - Candidato['Mediana_Esperada']) < 0.001):  # Tolerancia para decimales.
                    Verificacion['Valor_Correcto'] = True
                    Verificacion['Observaciones'] = '✅ Correcto'
                else:
                    Verificacion['Observaciones'] = f'❌ Esperaba {Candidato["Mediana_Esperada"]}, obtuvo {Valor_Después_Relleno}'
            
            elif pd.isna(Valor_Después_Relleno):
                Verificacion['Observaciones'] = '⚠️ No se rellenó (aún NaN)'
            elif pd.notna(Candidato['Valor_Antes_Relleno']):
                Verificacion['Observaciones'] = '⚠️ Ya tenía valor antes'
            
            Lista_Verificacion.append(Verificacion)
    
    # Crear DataFrame de verificación.
    df_Verificacion = pd.DataFrame(Lista_Verificacion)
    
    if len(df_Verificacion) > 0:
        # Estadísticas de verificación.
        Total_Candidatos = len(df_Verificacion)
        Rellenos_Exitosos = df_Verificacion['Relleno_Exitoso'].sum()
        Valores_Correctos = df_Verificacion['Valor_Correcto'].sum()
        
        print(f"📊 ESTADÍSTICAS DE VERIFICACIÓN:")
        print(f"   Total candidatos: {Total_Candidatos}")
        print(f"   Rellenos exitosos: {Rellenos_Exitosos}")
        print(f"   Valores correctos: {Valores_Correctos}")
        print(f"   Tasa de éxito: {(Valores_Correctos/Total_Candidatos)*100:.1f}%")
        
        # Mostrar casos problemáticos.
        Casos_Problemáticos = df_Verificacion[df_Verificacion['Valor_Correcto'] == False]
        
        if len(Casos_Problemáticos) > 0:
            print(f"\n⚠️ CASOS PROBLEMÁTICOS ({len(Casos_Problemáticos)}):")
            for _, Caso in Casos_Problemáticos.head(10).iterrows():  # Mostrar primeros 10.
                print(f"   ID {Caso['ID']} ({Caso['Categoria']}) - "
                      f"Item {Caso['Item_Numero']} {Caso['Tipo_Variable']} {Caso['Direccion_Rellenada']}")
                print(f"      {Caso['Observaciones']}")
        
        # Mostrar muestra de casos exitosos.
        Casos_Exitosos = df_Verificacion[df_Verificacion['Valor_Correcto'] == True]
        
        if len(Casos_Exitosos) > 0:
            print(f"\n✅ MUESTRA DE CASOS EXITOSOS:")
            for _, Caso in Casos_Exitosos.head(5).iterrows():  # Mostrar primeros 5.
                print(f"   ID {Caso['ID']} ({Caso['Categoria']}) - "
                      f"Item {Caso['Item_Numero']} {Caso['Tipo_Variable']} {Caso['Direccion_Rellenada']}")
                print(f"      Valor rellenado: {Caso['Valor_Después_Relleno']} "
                      f"(Mediana: {Caso['Mediana_Esperada']})")
        
        # Guardar DataFrame de verificación para revisión detallada.
        Nombre_Archivo_Verificacion = f'Verificacion_Granular_Relleno_{Nombre_df}.xlsx'
        df_Verificacion.to_excel(Nombre_Archivo_Verificacion, index=False)
        print(f"\n💾 Verificación detallada guardada en: {Nombre_Archivo_Verificacion}")

print("\n" + "="*80)
print("✅ VERIFICACIÓN GRANULAR COMPLETADA")
print("="*80)

In [32]:
# Exportar base para el control.
for Nombre_df, df in dfs_Finales.items():
    dfs_Finales[Nombre_df].head(50).to_excel(f'Controles/6. Relleno de medianas de IP asociados ({Nombre_df}).xlsx', index=False)